<a href="https://colab.research.google.com/github/LuciaMartinezGavier/compromiso-social-estudiantil/blob/main/parafrasis_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de paráfrasis en español


Elegir insecto
Opciones: babosa, oruga, hormigas, babositas

In [1]:
bug = 'hormiga'

Obtener los textos que se desean parafrasear

In [2]:
import requests

# url tiene el link a la descarga de un json con todos los párrafos
urls = {
  'babosa':   'https://drive.google.com/uc?export=download&id=1_0DHmdMzrr_0RCbc1-zxMsUNHZ_yTG4X',
  'oruga':    'https://drive.google.com/uc?export=download&id=1UBRHkv7s39ju6fmthJ7lSfQ41Fy6MF8C',
  'hormiga':  'https://drive.google.com/uc?export=download&id=1U63xv_DPTb4FAI9XKiJ9ly6au7FQhNKm',
  'babosita': 'https://drive.google.com/uc?export=download&id=1RzIe2ejobaGxN3m-nrA6p4hyLqdQxjI1'
}

try:
  bug = requests.get(urls[bug]).json()
except:
  print("An error ocurred")


Utilizar modelo para hacer paráfrasis

In [3]:
!pip install transformers==4.10.2
!pip install sentencepiece==0.1.96

In [4]:
# importar modelo de paráfrasis
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-paus-x-paraphrasing")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-paus-x-paraphrasing")
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("device ", device)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


device  cpu


In [6]:
from copy import deepcopy
parafrasis = {"que": [], "como": [], "habitat": [], "daño": [], "prevencion": []}

for topic in bug:
  for paragraph in bug[topic]:
    parafrasis[topic].append(paragraph)
    context = paragraph
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    text = context + " </s>"

    encoding = tokenizer.encode_plus(
        text,
        max_length = 500,
        padding = True,
        return_tensors = "pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=500,
        early_stopping=True,
        num_beams=15,
        num_return_sequences=3,
    )

    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens = True,
                                clean_up_tokenization_spaces = True)
        parafrasis[topic].append(sent)


In [7]:
for topic in parafrasis:
  for j in parafrasis[topic]:
    print(j)
  print("-------------------------")

Las hormigas son insectos que pertenecen a un grupo llamado insectos sociales, los cuales se caracterizan por vivir en comunidades organizadas con división de tareas. El tamaño de las colonias de hormigas pueden variar desde una docena hasta varios miles de ejemplares.
Las hormigas son insectos que depende su alimentación pueden ser nectívoras (se alimentan de soluciones azucaradas), granívoras (se alimentan de granos y semillas) y cortadoras (recolectan trozos de hojas que trasladan al nido para cultivar un hongo que sirve de alimento de adultos y larvas).
Las hormigas son insectos, que cumplen roles importantes en los ecosistemas, entre ellos, regular el crecimiento vegetal, reciclar nutrientes, ser controladores biológicos, polinizar, remover materia orgánica...
Las hormigas son insectos que pertenecen a un grupo llamado insectos sociales, los cuales se caracteriza por vivir en comunidades organizadas con división de tareas. El tamaño de las colonias pueden variar desde una Moisés h

Generar textos usando los párrafos creados. Un párrafo por tema.

In [8]:
from itertools import product
# combinar paráfrasis
#final_text = [t1 +"\n" + t2 +"\n" + t3 +"\n"+ t4 +"\n" + t5 for t1, t2, t3, t4, t5 in product(parafrasis["que"], parafrasis["como"], parafrasis["habitat"], parafrasis["daño"], parafrasis["prevencion"])]

# combinar párrafos madre
final_text = [
    t1 + "\n" + t2 + "\n" + t3 + "\n"+ t4 + "\n" + t5
    for t1, t2, t3, t4, t5 in product(bug["que"], bug["como"], bug["habitat"],
                                      bug["daño"], bug["prevencion"])
]

import random
for i in range(4):
  num = random.randint(0, len(final_text))
  print(final_text[num] + "\n\n")

Las hormigas son insectos que pertenecen a un grupo llamado insectos sociales, los cuales se caracterizan por vivir en comunidades organizadas con división de tareas. El tamaño de las colonias de hormigas pueden variar desde una docena hasta varios miles de ejemplares.
La longitud normal de una hormiga es de 1 a 5 mm, aunque se han llegado a descubrir hormigas de hasta 30 mm (no en la región patagónica).
Las hormigas habitan en casi todos los ecosistemas terrestres con excepción de la Antártida. Las especies que entran a nuestras casas en busca de protección o alimento son conocidas como 'Hormigas Urbanas'.
Las hormigas actúan como vectores de agentes patógenos. Dado su pequeño tamaño y su gran capacidad de desplazamiento, pueden propagar patógenos a los alimentos y a lugares sensibles como hospitales.
Como repelentes se puede colocar hojas de laurel alrededor de la despensa de alimentos. Las hormigas también son repelidas por el vinagre, se recomienda mezclar partes iguales de vinagre